In [ ]:
import gensim
import os

In [ ]:
import pandas as pd
devDs = pd.read_csv('/content/CT24_checkworthy_english_dev-test.tsv', sep = '\t')
testtDs = pd.read_csv('/content/CT24_checkworthy_english_dev.tsv', sep = '\t')
trainDs = pd.read_csv('/content/CT24_checkworthy_english_train.tsv', sep = '\t')
testDs = pd.read_csv('/content/CT24_checkworthy_english_test_gold.tsv', sep = '\t')

df = pd.concat([devDs, testtDs, trainDs], ignore_index = True)
df.head()

,Sentence_id,Text,class_label
0,37436,He's been a professor for a long time at a gre...,No
1,37440,There's no way they would give it up.,No
2,37463,They're able to charge women more for the same...,Yes
3,37470,"As far as a say is concerned, the people alrea...",No
4,37504,I am the Democratic Party right now.,No


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

story = []
for index, row in df.iterrows():
  text = row['Text']
  raw_sent = sent_tokenize(text)
  for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [ ]:
word2vecModel = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [ ]:
word2vecModel.build_vocab(story)

In [ ]:
word2vecModel.train(story, total_examples = word2vecModel.corpus_count, epochs = word2vecModel.epochs)

(2052537, 2843290)

In [ ]:
df2 = pd.read_csv('/content/CT24_checkworthy_english_test_gold.tsv', sep = '\t')
df2.head()

,Sentence_id,Text,class_label
0,35988,They said they were just going to get inspecto...,Yes
1,35991,"And from that point on, I've voted to -- I mov...",Yes
2,36029,I sit on the Senate Armed Services Committee.,No
3,36043,We need to depend on all of our tools -- diplo...,No
4,36063,And that is -- and I don't know if my colleagu...,No


In [ ]:
khela = 'as'
if khela in word2vecModel.wv:
  print("choltese")

choltese


In [ ]:
df['class_label'] = df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)
df2['class_label'] = df2['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [ ]:
!pip install keras_preprocessing

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
from keras.models import Sequential, Model
from keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional, GRU, MultiHeadAttention, LayerNormalization, Concatenate, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import f1_score
from nltk.tokenize import word_tokenize
from keras.callbacks import EarlyStopping
from imblearn.over_sampling import RandomOverSampler

def get_word_embedding_model(word_embedding_type):
    if word_embedding_type == "word2vec":
        word_embedding_model = word2vecModel
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")
    return word_embedding_model

def get_sentence_embedding(sentence, model, word_embedding_type):
    if word_embedding_type == "word2vec":
        tokens = word_tokenize(sentence.lower())
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        if not vectors:
            return np.zeros(model.vector_size)
        return np.mean(vectors, axis=0)
    else:
        raise ValueError("Unsupported word embedding type. Choose 'word2vec' or 'fasttext'.")

def create_embedding_matrix(word_index, word_embedding_model, embedding_dim):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word_embedding_model.wv:
            embedding_matrix[i] = word_embedding_model.wv[word]
        else:
            embedding_matrix[i] = np.zeros(embedding_dim)
    return embedding_matrix

def create_stacked_lstm_model(embedding_matrix, input_length, model_type):
    model = Sequential()
    model.add(Embedding(input_dim=embedding_matrix.shape[0],
                        output_dim=embedding_matrix.shape[1],
                        weights=[embedding_matrix],
                        input_length=input_length,
                        trainable=False))
    model.add(SpatialDropout1D(0.2))

    if model_type == "lstm":
        model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "bilstm":
        model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
        model.add(Bidirectional(LSTM(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True)))
    elif model_type == "gru":
        model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
        model.add(GRU(50, dropout=0.2, recurrent_dropout=0.2, return_sequences=True))
    else:
        raise ValueError("Unsupported model type. Choose 'lstm', 'bilstm', or 'gru'.")

    query = model.output
    key = model.output
    value = model.output

    attention_layer = MultiHeadAttention(num_heads=4, key_dim=50)
    attention_output = attention_layer(query=query, value=value, key=key)
    attention_output = LayerNormalization()(attention_output)

    # Concatenate LSTM output and attention output
    combined_output = Concatenate()([model.output, attention_output])

    # Apply GlobalAveragePooling1D to aggregate the sequence into a single vector
    combined_output = GlobalAveragePooling1D()(combined_output)

    combined_output = Dense(1, activation='sigmoid')(combined_output)

    final_model = Model(inputs=model.input, outputs=combined_output)
    final_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return final_model

def train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type):
    y_train = train_data["class_label"].values
    y_dev = dev_data["class_label"].values

    word_embedding_model = get_word_embedding_model(word_embedding_type)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(train_data["Text"].tolist() + dev_data["Text"].tolist())
    X_train_sequences = tokenizer.texts_to_sequences(train_data["Text"].tolist())
    X_dev_sequences = tokenizer.texts_to_sequences(dev_data["Text"].tolist())

    word_index = tokenizer.word_index
    embedding_matrix = create_embedding_matrix(word_index, word_embedding_model, word_embedding_model.vector_size)

    max_sequence_length = max(len(seq) for seq in X_train_sequences + X_dev_sequences)
    X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
    X_dev_padded = pad_sequences(X_dev_sequences, maxlen=max_sequence_length)

    # Handle data imbalance using random oversampling
    ros = RandomOverSampler(random_state=42)
    X_train_padded_resampled, y_train_resampled = ros.fit_resample(X_train_padded, y_train)

    stacked_lstm_model = create_stacked_lstm_model(embedding_matrix, max_sequence_length, model_type)

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    stacked_lstm_model.fit(X_train_padded_resampled, y_train_resampled, epochs=5, batch_size=32,
                           validation_data=(X_dev_padded, y_dev),
                           callbacks=[early_stopping])

    dev_predictions = (stacked_lstm_model.predict(X_dev_padded, verbose=0) > 0.5).astype("int32").flatten()
    y_dev = y_dev.flatten()
    f1score = f1_score(y_dev, dev_predictions)

    print(f"F1-score on Development Set: {f1score}")
    return stacked_lstm_model

In [ ]:
word_embedding_type = "word2vec"
model_type = "lstm"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
  18/1529 [..............................] - ETA: 45:31 - loss: 0.6755 - accuracy: 0.5799

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-15-720decf77eec>", line 5, in <cell line: 5>
    stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)
  File "<ipython-input-14-28798db5fb7a>", line 107, in train_and_evaluate_model
    stacked_lstm_model.fit(X_train_padded_resampled, y_train_resampled, epochs=5, batch_size=32,
  File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1807, in fit
    tmp_logs = self.train_function(iterator)
  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "/usr/local/li

TypeError: object of type 'NoneType' has no len()

In [ ]:
word_embedding_type = "word2vec"
model_type = "bilstm"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)

Epoch 1/5
1592/1592 [==============================] - 2031s 1s/step - loss: 0.4884 - accuracy: 0.7660 - val_loss: 0.5931 - val_accuracy: 0.6891
Epoch 2/5
1592/1592 [==============================] - 1967s 1s/step - loss: 0.4374 - accuracy: 0.7940 - val_loss: 0.4716 - val_accuracy: 0.7595
Epoch 3/5
1592/1592 [==============================] - 1975s 1s/step - loss: 0.3953 - accuracy: 0.8187 - val_loss: 0.4603 - val_accuracy: 0.7771
Epoch 4/5
1592/1592 [==============================] - 1958s 1s/step - loss: 0.3572 - accuracy: 0.8409 - val_loss: 0.4760 - val_accuracy: 0.7625
Epoch 5/5
1592/1592 [==============================] - 1928s 1s/step - loss: 0.3177 - accuracy: 0.8623 - val_loss: 0.5731 - val_accuracy: 0.7214
F1-score on Development Set: 0.5622119815668202


In [ ]:
word_embedding_type = "word2vec"
model_type = "gru"
train_data = df
dev_data = df2
stacked_lstm_model = train_and_evaluate_model(train_data, dev_data, word_embedding_type, model_type)